In [10]:
import cv2
import numpy as np

In [11]:
img = cv2.imread("img.jpg")
orig = img.copy()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

bw = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY_INV,
    25, 10
)


In [12]:
h, w = bw.shape

h_kernel = cv2.getStructuringElement(
    cv2.MORPH_RECT, (w // 15, 1)
)

horizontal = cv2.morphologyEx(bw, cv2.MORPH_OPEN, h_kernel)


In [13]:
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
    horizontal, connectivity=8
)

clean_horizontal = np.zeros_like(horizontal)

for i in range(1, num_labels):
    length = stats[i, cv2.CC_STAT_WIDTH]
    area = stats[i, cv2.CC_STAT_AREA]

    if length > w * 0.6 and area > 500:
        clean_horizontal[labels == i] = 255
v_kernel = cv2.getStructuringElement(
    cv2.MORPH_RECT, (1, h // 20)
)

vertical = cv2.morphologyEx(bw, cv2.MORPH_OPEN, v_kernel)
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
    vertical, connectivity=8
)

clean_vertical = np.zeros_like(vertical)

for i in range(1, num_labels):
    height = stats[i, cv2.CC_STAT_HEIGHT]
    area = stats[i, cv2.CC_STAT_AREA]

    if height > h * 0.6 and area > 500:
        clean_vertical[labels == i] = 255


In [14]:
grid = cv2.add(clean_horizontal, clean_vertical)


In [15]:
cv2.imshow("grid Image", grid)
cv2.waitKey(0)
cv2.destroyAllWindows()